In [1]:
from pathlib import Path
import subprocess

In [2]:
# directories
forcing_types = ['nldas','daymet','maurer']

config_dirs = {}
for ftype in forcing_types:
    config_dirs[ftype] = (Path(f'/home/gsnearing/projects/multi-inputs/multiple_forcing/configs/{ftype}'))

out_dirs = {}
for ftype in forcing_types:
    out_dirs[ftype] = (Path(f'{ftype}_runs'))

In [3]:
# load config files
config_files = {}
total_configs = 0
for ftype in forcing_types:
    config_files[ftype] = (list(config_dirs[ftype].glob('*.yml')))
    total_configs += len(config_files[ftype])
print(f'There are {total_configs} experiments.')

There are 30 experiments.


In [4]:
# optimizer hypers
max_model_runs = 1e4 # 1e3 # 1e5
dds_trials = 1
use_cores_frac = 1
algorithm = 'DDS'

In [5]:
# loop over experiments
for ftype in forcing_types:
    for f, config_file in enumerate(config_files[ftype]):
        
        # read basefile
        with open('run_job.slurm', 'r') as file:
            filedata = file.read()

        # replace experiment name
        filedata = filedata.replace('dummy', ftype)

        # replace config file
        filedata = filedata.replace("${1}", str(config_file))

        # replace number of model runs
        filedata = filedata.replace("${2}", str(int(max_model_runs)))

        # replace algorithm
        filedata = filedata.replace("${3}", algorithm)

        # replace output directory
        outdir = f'./results/{ftype}/'
        filedata = filedata.replace("${4}", outdir)

        # write to new config file
        conf = str(config_file).split('/')[-1].split('.')[0]
        job_file = Path(f"job_scripts/{ftype}_{conf}.slurm")
        with open(job_file, 'w') as file:
            file.write(filedata)


In [6]:
# submit all these jobs
job_files = list(Path('job_scripts').glob('**/*.slurm'))
print(f'There are {len(job_files)} experiments.')    

There are 30 experiments.


In [7]:
for file in job_files:
    run_cmd = f"sbatch {file}"
    log_file = f"log_files/{str(file).split('/')[-1].split('.')[0]}"
    with open(Path(log_file), 'w') as f:
        subprocess.Popen(run_cmd, stderr=subprocess.STDOUT, stdout=f, shell=True)